In [3]:
#THIS PROGRAM TAKES IN A CSV FILE WITH A LIST OF STOCK
#ITERATES THROUGH THOSE STOCKS AND GATHERS THE PREVIOUS MONTHS DATA AND SAVES IT TO CSV FILES
#USING BARCHART API

In [4]:
# IMPORTS #
import time
t0 = time.clock()

import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
import datetime as dt
from copy import copy
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)


In [5]:
# ================================================================== #
# datetime management

d = dt.date.today()
# ---------- Days ---------- 
l10 = d - 10 * BDay()
l21 = d - 21 * BDay()
l63 = d - 63 * BDay()
l252 = d - 252 * BDay()
# ---------- Years ---------- 
l252_x2 = d - 252 * 2 * BDay() 
l252_x3 = d - 252 * 3 * BDay() 
l252_x5 = d - 252 * 5 * BDay()
l252_x7 = d - 252 * 7 * BDay() 
l252_x10 = d - 252 * 10 * BDay() 
l252_x20 = d - 252 * 20 * BDay() 
l252_x25 = d - 252 * 25 * BDay()



In [6]:
# ================================================================== #
# filepath management

project_dir = r'/Users/jamesgalante/Documents/College/Semester 7/CPEG498/Uptick/Legacy Data/get past month data/' 
price_path = project_dir + r'Symbols/'

In [7]:
#BARCHART API
apikey = '0adbf00b462c1acca954a43d94279b92'
def construct_barChart_url(sym, start_date, freq, api_key=apikey):
    '''Function to construct barchart api url'''
    
    url = 'http://marketdata.websol.barchart.com/getHistory.csv?' +\
            'key={}&symbol={}&type={}&startDate={}'.format(api_key, sym, freq, start_date)
    return url

In [8]:
#get minute data for the past month for the stocks in the csv file
def get_minute_data():
    print("minute data:")
    '''Function to Retrieve <= 3 months of minute data for SP500 components'''
    
    # This is the required format for datetimes to access the API
    # You could make a function to translate datetime to this format
    start = '20100101' #start data doesnt matter as long as its more then a month older
    #end = d
    freq = 'minutes'    
    prices = {}
    symbol_count = len(syms)
    N = copy(symbol_count)
    try:
        for i, sym in syms.iterrows():
            
            print(sym["STOCKS"])
            api_url = construct_barChart_url(sym["STOCKS"], start, freq, api_key=apikey)
            print(api_url)
            try:
                csvfile = pd.read_csv(api_url, parse_dates=['timestamp'])
                csvfile.set_index('timestamp', inplace=True)
                prices[sym["STOCKS"]] = csvfile
            except:
                continue
            N -= 1
            pct_total_left = (N/symbol_count)
            print('{}..[done] | {} of {} symbols collected | percent remaining: {:>.2%}'.format(\
                                                                sym, i, symbol_count, pct_total_left)) 
    except Exception as e: 
        print(e)
    finally:
        pass
    px = pd.Panel.from_dict(prices)

    return px



In [9]:
# Read the xls file with the stocks we want to gather data from #
spy_components = pd.read_excel(price_path + 'stock_symbols.xls', header=0) 
syms = spy_components.dropna()
print(syms)
print()

pxx = get_minute_data()

  STOCKS
0   AAPL
1   MSFT
2   AMZN
3     VZ
4   TSLA

minute data:
AAPL
http://marketdata.websol.barchart.com/getHistory.csv?key=0adbf00b462c1acca954a43d94279b92&symbol=AAPL&type=minutes&startDate=20100101
STOCKS    AAPL
Name: 0, dtype: object..[done] | 0 of 5 symbols collected | percent remaining: 80.00%
MSFT
http://marketdata.websol.barchart.com/getHistory.csv?key=0adbf00b462c1acca954a43d94279b92&symbol=MSFT&type=minutes&startDate=20100101
STOCKS    MSFT
Name: 1, dtype: object..[done] | 1 of 5 symbols collected | percent remaining: 60.00%
AMZN
http://marketdata.websol.barchart.com/getHistory.csv?key=0adbf00b462c1acca954a43d94279b92&symbol=AMZN&type=minutes&startDate=20100101
STOCKS    AMZN
Name: 2, dtype: object..[done] | 2 of 5 symbols collected | percent remaining: 40.00%
VZ
http://marketdata.websol.barchart.com/getHistory.csv?key=0adbf00b462c1acca954a43d94279b92&symbol=VZ&type=minutes&startDate=20100101
STOCKS    VZ
Name: 3, dtype: object..[done] | 3 of 5 symbols collected | perc

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.



In [10]:
# convert timestamps to EST
pxx.major_axis = pxx.major_axis.tz_localize('utc').tz_convert('US/Eastern')

In [24]:
#SAVE FILES
print(pxx)
outputPath = project_dir + 'output files/'
for i, sym in syms.iterrows():
    pxx[sym["STOCKS"]].to_csv(outputPath + sym["STOCKS"]+'.csv')

#pxx['AAPL'].to_csv(outputPath + 'AAPL.csv')
#pxx['MSFT'].to_csv(outputPath + 'MSFT.csv')
##pxx['AMZN'].to_csv(outputPath + 'AMZN.csv')
#pxx['VZ'].to_csv(outputPath + 'VZ.csv')
#pxx['TSLA'].to_csv(outputPath + 'TSLA.csv')

<class 'pandas.core.panel.Panel'>
Dimensions: 5 (items) x 8211 (major_axis) x 7 (minor_axis)
Items axis: AAPL to VZ
Major_axis axis: 2018-09-07 09:30:00-04:00 to 2018-10-05 16:00:00-04:00
Minor_axis axis: symbol to volume
AAPL
MSFT
AMZN
VZ
TSLA
